# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [ ]:
import os, re, time, copy
from html import escape
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [ ]:
# Load the binary program file
program_file = "challenge.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
vm_program = open(file_path, "rb").read()

In [ ]:
class SynacorConsole:
    track_backwards = {"north":"south", "south":"north", "east":"west","west":"east"}

    def __init__(self, software, visualize: bool = True):
        self.software = software
        self.visualize = visualize
        self.console = self.reset_console()

    def reset_console(self):
        console = VirtualMachine(self.software)
        self.overall_commands = []
        self.challenge_codes = {}
        self.valid_codes = set()
        self.game_map = {}
        self.code_no = 0
        return console

    @staticmethod
    def md5_hash(code: str) -> str:
        import hashlib
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    def __extract_codes(self, text):
        """Extract mixed-case challenge codes and store them with their MD5 hashes."""
        codes = re.findall(r"\b(?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*\b", text)

        new_codes = []
        for code in codes:
            if code not in self.valid_codes:
                self.code_no += 1
                self.challenge_codes[self.code_no] = (code, self.md5_hash(code))
                new_codes.append(code)
                self.valid_codes.add(code)
        return new_codes

    def __format_terminal(self, text, codes_color):
        """Highlight mixed-case challenge codes in HTML."""
        text = escape(text)
        return re.sub(
            r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
            rf"<span style='color:{codes_color}'>\1</span>",
            text
        )

    def display_terminal(self, text):
        """Format and display terminal output with highlighted codes."""
        self.color_dict = {
            "background": "#282828", "terminal": "#33FF00", "codes": "#FFB000"
        }
        longest_line = max(len(line) for line in text.split('\n'))

        html = (
            f"<div style='background-color: {self.color_dict['background']}; "
            f"width: {longest_line + 2}ch; padding: 0.75ex;'>"
            f"<pre style='background-color: {self.color_dict['background']}; "
            f"color: {self.color_dict['terminal']}; margin: 0; font-family: monospace; "
            f"width: {longest_line + 2}ch;'>"
            + self.__format_terminal(text, self.color_dict['codes'])
            + "</pre></div>"
        )

        display(HTML(html))

    def play_game_manually(self, actions = []):
        if actions:
            self.overall_commands.extend(actions)
            print("Game Inputs:", actions)
        full_terminal = self.console.run_computer(actions)
        current_terminal = full_terminal[-1]
        self.display_terminal(current_terminal)
        self.__extract_codes(current_terminal)
        return self.overall_commands

    def __parse_game_state(self, terminal):
        lines = terminal.splitlines()
        things, exits, location = ([], [], "")

        current_section = None
        for line in lines:
            line = line.strip()
            if line.startswith("==") and line.endswith("=="):
                location = line.strip("= ").strip()
            elif line.startswith("Things of interest here:"):
                current_section = "things"
            elif line.startswith("There") and "exit" in line:
                current_section = "exits"
            elif line.startswith("-") and current_section:
                item = line[2:].strip()
                if current_section == "things":
                    things.append(item)
                elif current_section == "exits":
                    exits.append(item)
            else:
                current_section = None

        return location, things, exits

    def __generate_commands(self, current_state, commands_list = []):
        location, all_items, exits = self.__parse_game_state(current_state)
        self.game_map[location] = {"things":all_items, "exits":exits}
        if all_items:
            for item in all_items:
                commands_list.append(f"take {item}")
                # commands_list.append(f"use {item}")
        commands_list.append(f"{exits[0]}")
        # commands_list.append("help")
        return commands_list

    def play_game(self, actions = []):
        steps = 0
        while True:
            if actions:
                self.overall_commands.extend(actions)
                print("Game Inputs:", actions)
            full_terminal = self.console.run_computer(actions)
            current_terminal = full_terminal[-1]
            self.__extract_codes(current_terminal)
            self.display_terminal(current_terminal)
            if steps >= 10:
                break
            actions = self.__generate_commands(current_terminal, [])
            steps += 1
        return self.overall_commands

In [ ]:
console = SynacorConsole(vm_program)
command_list = console.play_game()

In [ ]:
for code_no, code_group in console.challenge_codes.items():
    print(code_no, code_group)
print(f"Execution Time = {time.time() - start_time:.5f}s")